# Gaussian Target Practice

In [1]:
using Pkg
Pkg.activate("..")

  Activating project at `~/Documents/PhD/MicroCanonicalHMC.jl/examples`


In [2]:
# The statistical inference frame-work we will use
using Turing
using LinearAlgebra
using Random
using PyPlot

using Revise
using MicroCanonicalHMC

[ Info: Precompiling Turing [fce5fe82-541a-59a6-adf8-730c64b5f9a0]
Precompiling TuringOptimExt
  ✓ Manifolds → ManifoldsRecipesBaseExt
  ✓ Turing → TuringOptimExt
  2 dependencies successfully precompiled in 131 seconds. 323 already precompiled.
[ Info: Precompiling TuringOptimExt [cd2420fc-8d31-5c39-8d98-0365cfcf7d6e]
┌ Warning: Module Turing with build ID fafbfcfd-bd36-5380-0000-09c5b41257de is missing from the cache.
│ This may mean Turing [fce5fe82-541a-59a6-adf8-730c64b5f9a0] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1948
[ Info: Skipping precompilation since __precompile__(false). Importing TuringOptimExt [cd2420fc-8d31-5c39-8d98-0365cfcf7d6e].
Precompiling PyPlot
  ✓ PyPlot
  1 dependency successfully precompiled in 1 seconds. 17 already precompiled.
[ Info: Precompiling PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
Precompiling SciMLBasePyCallExt
  ✓ SciMLBase → SciMLBasePyCallExt
  1 dependency successfully precompiled in 8 seco

In [3]:
@model function model_Gaussian(rt)
    # Priors
    μ ~ Normal(0.3, 0.5)
    σ ~ Normal(log(0.2), 3)
    # Data
    rt ~ filldist(Normal(μ, exp(σ)), length(rt))
end

fit = model_Gaussian(rand(100))

LoadError: ParseError:
[90m# Error @ [0;0m]8;;file:///home/jaimerz/Documents/PhD/MicroCanonicalHMC.jl/examples/Gaussian/In[3]#8:1\[90mIn[3]:8:1[0;0m]8;;\
    end
[48;2;120;70;70mend[0;0m
[90m└─┘ ── [0;0m[91minvalid identifier[0;0m

## MCHMC

In [ ]:
samples_MCHMC = sample(fit, externalsampler(MCHMC(200, 0.01; adaptive=true)), 200)

In [ ]:
theta_mchmc = [samples.value.data[i, 1, :][1] for i in axes(samples.value.data)[1]]
x10_mchmc = [samples.value.data[i, 10+1, :][1] for i in axes(samples.value.data)[1]];

## NUTS

In [ ]:
samples_NUTS = sample(fit, NUTS(), 200)  # Works

## Comp

In [ ]:
fig, axis = plt.subplots(2, 2, figsize=(8,8))
fig.suptitle("Neal's Funnel Comp.", fontsize=16)

fig.subplots_adjust(hspace=0)
fig.subplots_adjust(wspace=0)

axis[1,1].hist(x10_mchmc, bins=100, density=true, range=[-6,2], alpha = 0.3, label="MCHMC")
axis[1,1].hist(x10_hmc, bins=100, density=true, range=[-6,2], alpha = 0.3, label="NUTS")
axis[1,1].hist(true_x10_hmc, bins=100, density=true, range=[-6,2], alpha = 0.3, label="NUTS - Truth")
axis[1,1].legend()
axis[1,1].set_yticks([])

axis[2,2].hist(theta_mchmc, bins=100, density=true, orientation="horizontal", range=[-6, 2], alpha = 0.3)
axis[2,2].hist(theta_hmc, bins=100, density=true, orientation="horizontal", range=[-6, 2], alpha = 0.3)
axis[2,2].hist(true_theta_hmc, bins=100, density=true, orientation="horizontal", range=[-6,2], alpha = 0.3)
axis[2,2].set_xticks([])
axis[2,2].set_yticks([])

axis[1,2].hist2d(true_x10_hmc, true_theta_hmc, bins=100, range=[[-6,2],[-4, 2]])
axis[1,2].set_xlabel("x10")
axis[1,2].set_ylabel("theta")
axis[1,2].set_title("NUTS")

axis[2,1].hist2d(x10_mchmc, theta_mchmc, bins=100, range=[[-6,2],[-4, 2]])
axis[2,1].set_xlabel("x10")
axis[2,1].set_ylabel("theta")
axis[2,1].set_title("MCHMC")